# vision text data NLP 
Tony 08/12/2023

Input:
- raw data from my detect_road module
- chatGPT generated analysis result

Output:
- bbox json of specific video 
- risk json of specific video

In [1]:
# Let's read the content of the uploaded file to understand the key frame information
file_path = "raw_data/Info_Video_0194.txt"

with open(file_path, 'r') as file:
    key_frame_data = file.read()

# Displaying the first 1000 characters to get an overview of the content
key_frame_data[:1000]


'INFO of frame 0001:\nroad 0 is at middle_down\nsidewalk 1 is at right_down\nperson 0 is at middle_down\nThe [distance,angle] of person 0 is: [very close,slightly to the right]\ncar 1 is at right_down\n<V> car 1 is at [1113.8364  692.2208 1457.0139  925.9918] <VE> \nperson 2 is at left_down\nThe [distance,angle] of person 2 is: [very close,None]\nperson 3 is at left_down\nThe [distance,angle] of person 3 is: [very close,None]\nperson 4 is at middle_down\nThe [distance,angle] of person 4 is: [very close,None]\nPerson 0 is on the road 0, sidewalk 1, his/her bbox is[1172.3464   659.84393 1211.4929   705.81586]\nPerson 2 is on the road 0, sidewalk 1, his/her bbox is[288.8023  655.01025 308.0401  702.405  ]\nPerson 3 is on the road 0, his/her bbox is[620.4963  704.68494 631.8676  736.1472 ]\nPerson 4 is not on any detected surface, his/her bbox is[643.36334 709.2158  660.0417  760.1974 ]\nINFO of frame 0002:\nroad 0 is at middle_down\ncar 0 is at right_down\n<V> car 0 is at [1342.1464   697

In [8]:
# Let's read the content of the LLM generated file and load into JSON
file_path = "LLM_data/Result_Video_0194.txt"

with open(file_path, 'r') as file:
    result_data = file.read()

# Displaying the first 1000 characters to get an overview of the content
result_data[:1000]


'### Frame 0001 Analysis\n\n**Summary of Key Frame Information:**\n- **Road Position:** The road (road 0) is located in the middle-down region of the frame.\n- **Sidewalk Position:** The sidewalk (sidewalk 1) is located in the right-down region of the frame.\n- **People Positions and Distances:**\n  - **Person 0:** Located at middle-down, very close to the dashcam, slightly to the right. On road 0, sidewalk 1.\n  - **Person 2:** Located at left-down, very close to the dashcam. On road 0, sidewalk 1.\n  - **Person 3:** Located at left-down, very close to the dashcam. On road 0.\n  - **Person 4:** Located at middle-down, very close to the dashcam. Not on any detected surface.\n\n**Potential Risks:**\n- **Persons on Road and Sidewalk:** Persons 0, 2, 3 are on the road or sidewalk, posing a potential risk of collision.\n- **Close Proximity to Dashcam:** All detected persons are very close to the dashcam, indicating a crowded and potentially dangerous situation.\n- **Non-Detected Surfaces:*

数据筛选清单处理


In [6]:
# Given content of the "video_human_selection.txt" file
content = """
b1c9c847-3bda4659     1 10 12  19
b1d10d08-c35503b8     7 10 83 90
b1d22ed6-f1cac061       12 29 38 36 35 52 51 66 67 68  77 78 100 107 104 105 157 153 162 163 166 173 181 186 176 160 194 197 170 186 234 236 250 251  295 304 309 295 324 293 357 368 413 415     
b1dac7f7-6b2e0382      1 8 49
b1f4491b-33824f31       4 36  43 44 77  119 120 169 166 170 167 185 209 235
b2ae0446-4b0bfda7      none
b2be7200-b6f7fe0a       1 2 6 13  22 26 29 28 31 34 39 66 67
b2bee3e1-80c787bd      2 6  12 
b2d22b2f-8302eb61       18  44 46 47  51 55  62 63 
b2d502aa-f0b28e3e      12 16  29 30 33
b4542860-0b880bb4      93  240 270  265  295 314 307
cd09a73f-5f6b9212        1  12
cd17ff29-f393274e         1 2 3 4  25 31  37   47 49 68
cd26264b-22001629       2 8  19 47 24 26  12 13 57 9 43 18  46 45  18 43 45 24 48 96 97 107 109 127 151 153
"""
import json
# Process content and convert to desired JSON structure
video_selection = {}
for line in content.strip().split("\n"):
    parts = line.split()
    video_name = parts[0]
    if parts[1] == "none":
        ids = []
    else:
        ids = [int(pid) for pid in parts[1:]]
    video_selection[video_name] = ids

def save_as_json(json_data, json_file_path):
    """Save data as a JSON file."""
    with open(json_file_path, 'w') as file:
        json.dump(data, file, indent=4)

save_as_json(video_selection, "human_select.json")


In [7]:


# Process content and convert to desired JSON structure
video_selection = {}
for line in content.strip().split("\n"):
    parts = line.split()
    video_name = parts[0]
    if parts[1] == "none":
        ids = []
        persons_data = {}
    else:
        ids = [int(pid) for pid in parts[1:]]
        persons_data = {str(pid): {"starting_frame": None, "ending_frame": None} for pid in ids}
    
    video_data = {
        "ids": ids
    }
    video_data.update(persons_data)
    video_selection[video_name] = video_data


save_as_json(video_selection, "human_select_interval.json")


In [3]:
def is_small_bbox(bbox):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1

    # You can adjust these thresholds as needed
    width_threshold = 5.0
    height_threshold = 5.0

    return width < width_threshold or height < height_threshold


In [4]:
from collections import defaultdict
import re
# Function to parse the raw json_data and extract the relevant information

# Person 0 is on the road 0, sidewalk 1,his/her bbox is [1148.4463   625.3968  1183.7595   663.28143]
# <V> car 1 is at [ 28.309494 712.38245  192.96992  777.90173 ] <VE> 
def parse_raw_data(json_data):
    frames = defaultdict(lambda: {
        "person": defaultdict(lambda: [None, None, None, None]), # Default bounding box for persons
        "car": defaultdict(lambda: [None, None, None, None])    # Default bounding box for cars
    })
    current_frame = None

    for line in json_data.split("\n"):
        if line.startswith("INFO of"):
            current_frame = line.split(":")[0].split()[-1]

        elif "Person" in line and "bbox" in line:
            person_id, bbox_str = re.match(r"Person (\d+) .* bbox is(\[.*?\])", line).groups()
            bbox_str = bbox_str.strip('[]')
            bbox = [float(value) for value in bbox_str.split()]
            if is_small_bbox(bbox):
                bbox =[ None, None, None, None]

            frames[current_frame]['person'][int(person_id)] = (bbox)


        elif "Person" in line and "detected" in line:
            person_id = re.match(r"Person (\d+) is not", line).group(1)

            frames[current_frame]['person'][int(person_id)] = [None,None,None,None]
        elif "<V> car" in line and "<VE>" in line:
            car_id, bbox_str = re.match(r"<V> car (\d+) is at (\[.*?\]) <VE>", line).groups()
            bbox = [float(value) for value in bbox_str.strip('[]').split()]
            
            if is_small_bbox(bbox):
                bbox =[ None, None, None, None]
            frames[current_frame]['car'][int(car_id)] = bbox


    return frames

# Reading the raw json_data file
file_path = "raw_data/Info_Video_0194.txt"

with open(file_path, 'r') as file:
    raw_data_content = file.read()
# Parsing the raw json_data to extract the required information
parsed_data = parse_raw_data(raw_data_content)
parsed_data


defaultdict(<function __main__.parse_raw_data.<locals>.<lambda>()>,
            {'0001': {'person': defaultdict(<function __main__.parse_raw_data.<locals>.<lambda>.<locals>.<lambda>()>,
                          {0: [1172.3464, 659.84393, 1211.4929, 705.81586],
                           2: [288.8023, 655.01025, 308.0401, 702.405],
                           3: [620.4963, 704.68494, 631.8676, 736.1472],
                           4: [643.36334, 709.2158, 660.0417, 760.1974]}),
              'car': defaultdict(<function __main__.parse_raw_data.<locals>.<lambda>.<locals>.<lambda>()>,
                          {1: [1113.8364, 692.2208, 1457.0139, 925.9918]})},
             '0002': {'person': defaultdict(<function __main__.parse_raw_data.<locals>.<lambda>.<locals>.<lambda>()>,
                          {1: [633.6151, 712.92944, 654.3307, 770.7612]}),
              'car': defaultdict(<function __main__.parse_raw_data.<locals>.<lambda>.<locals>.<lambda>()>,
                          {0: [134

In [5]:
import json

def fill_missing_frames(json_file_path):
    json_file_path = "JSON_data/" + json_file_path
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

    # Find the maximum frame number
    max_frame = int(max(json_data.keys(), key=lambda x: int(x)))

    # Create an empty frame structure
    empty_frame = {
        "person": {},
        "car": {}
    }

    # Fill in the missing frames with the empty frame structure
    for frame_number in range(max_frame + 1):
        frame_key = str(frame_number).zfill(4)
        if frame_key not in json_data:
            json_data[frame_key] = empty_frame

    # Write the filled json_data back to the JSON file
    with open(json_file_path, 'w') as file:
        json.dump(json_data, file, indent=4)

# Example usage
fill_missing_frames('Bbox_Video_274e.json')


In [6]:
import json
import os

def save_to_json(parsed_data, filename):
    # Create a directory named "JSON_data" if it doesn't exist
    os.makedirs('JSON_data', exist_ok=True)

    # Define the path
    path = os.path.join('JSON_data', filename)

    # Convert the dictionary to a JSON string and write it to the file
    with open(path, 'w') as file:
        json.dump(parsed_data, file, indent=4)

    print(f"json_data has been saved to {path}")



# Usage example
parsed_data = parse_raw_data(raw_data_content)
filename = 'Bbox_Video_0194.json'
save_to_json(parsed_data, filename)


Data has been saved to JSON_data\Bbox_Video_0194.json


### raw_data -> bbox
now I have a folder of the files, named 'raw_data/Info_Video_xxxx.txt'. I want you to write a script to go over all these txt file, and :
- data = read into str
- parsed_data = parse_raw_data(data)
- save_to_json(parsed_data,filename)
For example:
# Reading the raw data file
file_path = "raw_data/Info_Video_0194.txt"

with open(file_path, 'r') as file:
    raw_data_content = file.read()
# Parsing the raw data to extract the required information
parsed_data = parse_raw_data(raw_data_content)
filename = 'Bbox_Video_0194.json'
save_to_json(parsed_data, filename)

give me a script


In [7]:

# Directory containing the raw data files
raw_data_dir = "raw_data"

# Iterate through each file in the raw data directory
for filename in os.listdir(raw_data_dir):
    if filename.endswith(".txt"):
        # Construct the full path of the file
        file_path = os.path.join(raw_data_dir, filename)

        # Read the raw data from the file
        with open(file_path, 'r') as file:
            raw_data_content = file.read()

        # Parse the raw data
        parsed_data = parse_raw_data(raw_data_content)

        # Generate the output JSON filename
        json_filename = 'Bbox' + filename.replace("Info", "").replace(".txt", ".json")

        # Save the parsed data to a JSON file
        save_to_json(parsed_data, json_filename)
        
        fill_missing_frames(json_filename)


print("Processing completed!")


Data has been saved to JSON_data\Bbox_Video_0194.json
Data has been saved to JSON_data\Bbox_Video_0382.json
Data has been saved to JSON_data\Bbox_Video_03b8.json
Data has been saved to JSON_data\Bbox_Video_0bb4.json
Data has been saved to JSON_data\Bbox_Video_1629.json
Data has been saved to JSON_data\Bbox_Video_274e.json
Data has been saved to JSON_data\Bbox_Video_4659.json
Data has been saved to JSON_data\Bbox_Video_4f31.json
Data has been saved to JSON_data\Bbox_Video_87bd.json


Data has been saved to JSON_data\Bbox_Video_8e3e.json
Data has been saved to JSON_data\Bbox_Video_9212.json
Data has been saved to JSON_data\Bbox_Video_bd99.json
Data has been saved to JSON_data\Bbox_Video_c061.json
Data has been saved to JSON_data\Bbox_Video_eb61.json
Data has been saved to JSON_data\Bbox_Video_fda7.json
Data has been saved to JSON_data\Bbox_Video_fe0a.json
Processing completed!


In [8]:
import os
def preprocess_text(text):
    # Replace bold markdown with unique markers
    text = text.replace('**', '||') # Using '||' as a unique marker for bold text
    
    # Adding distinct delimiters for each person's information
    lines = text.split('\n')
    processed_lines = []
    for line in lines:
        if 'Person' in line and ':' in line:
            processed_lines.append('<PERSON_INFO>') # Start of person's information
        processed_lines.append(line)
        if line.strip().endswith(')'):
            processed_lines.append('</PERSON_INFO>') # End of person's information
            
    processed_text = '\n'.join(processed_lines)
    return processed_text


def store_preprocessed_text(filename, preprocessed_text, folder='prep_data'):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # Construct the new filename based on the provided name
    base_name = os.path.basename(filename)
    new_name = f"Prep_{base_name.replace('.txt', '.md')}"
    file_path = os.path.join(folder, new_name)

    # Write the preprocessed text to the new file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(preprocessed_text)
    
    print(f"Preprocessed text saved to: {file_path}")

# Example usage
filename = "Info_Video_0194.txt"
preprocessed_text = preprocess_text(result_data)
store_preprocessed_text(filename, preprocessed_text)


Preprocessed text saved to: prep_data\Prep_Info_Video_0194.md


In [9]:

def parse_text(text, bbox_data):
    # Initialize the result
    result = {}

    # Splitting the text into sections based on headers
    sections = text.split('###')

    # Iterate through the sections
    for section in sections:
        if "Risk Evaluation" in section:
            # Extracting frame number
            frame_number_match = re.search(r"Frame (\d+)", section)

            
            if frame_number_match:
                
                # Extracting the last paragraph as text summary
                text_summary = section.strip().split("\n\n")[-1]
                
                frame_number = frame_number_match.group(1)
                result[f"Frame{frame_number.zfill(4)}"] = {
                    "Time": f"{float(frame_number) /2}s",
                    "Text": text_summary,  # place holder here
                    "Pedestrian": {}
                }
           
                # Extracting risk evaluations for persons
                person_info_blocks = section.split('<PERSON_INFO>')[1:]
                for person_info_block in person_info_blocks:

                    person_info = person_info_block.split('</PERSON_INFO>')[0]
                    # person_match = re.search(r"Person (\d+): ([A-Za-z]+)", person_info)
                    person_match = re.search(r"\|\|Person (\d+):\|\| ([A-Za-z]+)", person_info)

                    if person_match:
                        person, risk_level = person_match.groups()
                        risk_level = risk_level.strip()
                        # print("risk_level",risk_level)
                        # Getting the bounding box data
                        bbox = bbox_data[frame_number.zfill(4)]['person'][str(person)]
                        
                        # Storing the extracted information
                        result[f"Frame{frame_number.zfill(4)}"]["Pedestrian"][person] = (risk_level, bbox)

    return result



# Preprocessing the text
preprocessed_text = preprocess_text(result_data)

# Parsing the text into JSON
json_data = parse_text(preprocessed_text, parsed_data)


json_data

{'Frame0001': {'Time': '0.5s',
  'Text': 'The situation in Frame 0001 indicates potential risks due to the presence of persons on the road and sidewalk, their close proximity to the dashcam, and their angles near the extremes of the field of view. Additionally, the presence of persons not on any detected surface raises concerns about their unexpected positions or movements. Immediate attention and caution are advised to prevent potential collisions.',
  'Pedestrian': {'0': ('High', [None, None, None, None]),
   '1': ('High', [None, None, None, None]),
   '2': ('High', [None, None, None, None]),
   '3': ('Medium', [None, None, None, None]),
   '4': ('Medium', [None, None, None, None]),
   '5': ('Medium', [None, None, None, None])}},
 'Frame0002': {'Time': '1.0s',
  'Text': 'The situation in Frame 0002 is more critical compared to the previous frame, as more persons are now detected on the road. The high-risk evaluation for Persons 0 through 4 is due to their positions on the road or sid

In [10]:
risk,bbox = json_data["Frame0005"]['Pedestrian']['0']
print(risk)
print(bbox)

High
[None, None, None, None]


In [11]:
save_to_json(json_data,"risk_evaluation_0194.json")

Data has been saved to JSON_data\risk_evaluation_0194.json


In [12]:

def load_box_data(json_filename):
    with open(json_filename, 'r') as file:
        box_data = json.load(file)
    return box_data


In [13]:
from pathlib import Path

result_data_dir = "LLM_data"

for filename in os.listdir(result_data_dir):
    if filename.endswith(".txt"):

        file_path = os.path.join(result_data_dir, filename)

        json_filename = 'risk_evaluation' + filename.replace("Result_Video", "").replace(".txt", ".json")

        output_path = Path('JSON_data/'+json_filename)
        if output_path.exists():
            print(f"Already handled {output_path}")
            continue

        with open(file_path, 'r') as file:
            result_data = file.read()
            file.close()

        pre_file_path = 'prep_data/' + filename
        preprocessed_text = preprocess_text(result_data)
        store_preprocessed_text(pre_file_path, preprocessed_text)

        bbox_json = 'JSON_data/Bbox_Video' + filename.replace("Result_Video", "").replace(".txt", ".json")
        print(bbox_json)
        # Load the corresponding box_data from the JSON file
        box_data = load_box_data(bbox_json)
        
        json_data = parse_text(preprocessed_text, box_data) # need to read form box json

        save_to_json(json_data, json_filename)

print("Processing completed!")


Already handled JSON_data\risk_evaluation_0001.json
Already handled JSON_data\risk_evaluation_0003.json
Already handled JSON_data\risk_evaluation_0055.json
Already handled JSON_data\risk_evaluation_0056.json
Already handled JSON_data\risk_evaluation_0057.json
Already handled JSON_data\risk_evaluation_0194.json
Already handled JSON_data\risk_evaluation_0310.json
Already handled JSON_data\risk_evaluation_0313.json
Already handled JSON_data\risk_evaluation_0333.json
Preprocessed text saved to: prep_data\Prep_Result_Video_0343.md
JSON_data/Bbox_Video_0343.json


KeyError: '0'

I wrote a code to parse the information from the raw_data, but there are two bug within it:
1. program don't show person bbox because the raw data don't have it, I want you to record as [None, None, None, None]
2. the key of bbox is string, I want it to be int, because it may be more convinent to search

Debug now



非常好，那么让我们处理另外一个问题
我现在需要将另外一个名为：all_video_name.json转化为自然语言，但是我只需要其中每个帧的行人数据

In [4]:
def read_mot_conf(lines):
    """
    Read the MOT file and return a dictionary mapping frame and id to confidence.
    :param mot_file: Path to the MOT formatted file.
    :return: Nested dictionary with frame and id as keys and confidence as value.
    """
    conf_data = {}
    for line in lines:
        parts = line.strip().split(',')
        frame, obj_id, _, _, _, _, conf, _, _, _ = map(float, parts)
        frame = int(frame)
        obj_id = int(obj_id)
        if frame not in conf_data:
            conf_data[frame] = {}
        conf_data[frame][obj_id] = conf

    return conf_data

In [6]:
import json

def transform_all_json_to_txt(video_name):
    # Load the JSON file
    json_filename = f'./JSON_data/all_json/all_{video_name}.json'
    with open(json_filename, 'r') as f:
        json_data = json.load(f)
    mot_filename = f'./raw_data/p_mot_list/{video_name}.txt'
    with open(mot_filename, 'r') as f:
        lines = f.readlines()

    conf_data = read_mot_conf(lines)

    output = []

    # Convert to natural language description
    for frame, frame_data in json_data.items():
        people = frame_data.get("person", {})
        output.append(f"### frame {frame}")

        person_ids = list(people.keys())
        if not person_ids:
            output.append("no detected person\n")
            continue
        person_ids_str = [int(pid) for pid in people.keys()]  # Convert keys (person IDs) to integers
        output.append(f"people detected: {person_ids_str}")

        # For each person, display bbox
        for pid, bbox in people.items():
            rounded_bbox = [round(coord, 1) for coord in bbox]
            output.append(f"{pid} is at {rounded_bbox} ")

        # Dummy logic for left/right of car based on bbox x-axis value (assuming bbox format is [x1, y1, x2, y2])
        right_of_car = [int(pid) for pid, bbox in people.items() if bbox[0] > 960]  # using 960 as video shape threshold
        left_of_car = [int(pid) for pid in person_ids if pid not in right_of_car]
        output.append(f"On right of our car, there are: {right_of_car}")
        output.append(f"On the left of our car, there are {left_of_car}")

        # Logic for high/low confidence using real confidence data
        high_confidence = [pid for pid in person_ids if float(conf_data[int(frame)][int(pid)]) >= 0.7]
        low_confidence = [pid for pid in person_ids if float(conf_data[int(frame)][int(pid)]) <= 0.3]
       


        output.append(f"high confidence: {', '.join(high_confidence) if high_confidence else 'none'}")
        output.append(f"low confidence: {', '.join(low_confidence) if low_confidence else 'none'} ")
        output.append("")  # For an empty line between frames

    # Save to an output file
    with open(f'./LLM_data/0829_J2T/{video_name}.txt', 'w') as f:
        f.write("\n".join(output))
transform_all_json_to_txt('video_0194')

In [ ]:
unproceessed_list = ['b4542860-0b880bb4','cd09a73f-5f6b9212','cd26264b-22001629']

for name in unproceessed_list:
    transform_all_json_to_txt(name)